# The Urbanizational Development of New York City 

***The City That Never Sleeps - From Seed To Apple***


# Introdution 

The notebook is a behind-the-scenes look at the data wrangling behind the story on *The Urbanizational Evolution of New York City - From Seed to Apple* presented on https://esbenbl.github.io/


The structure of the notebook is:
1. [Motivation](#Motivation)
2. [Basic Statistics](#base_stats)
    - [Dataset of the buildings of New York City](#Dataset_of_the_buildings)
    - [Dataset of the population evolution of New York](#Dataset_of_the_population)
    - [Dataset on ethnicity population in NYC neighboorhoods between 20XX and 2020](Dataset_ethnicitiy)
3. [Data Analysis](#Data_Analysis)
4. [Genre](#Genre)
5. [Visualization](#Visualization)
    - [TITEL Figure 1](#Figur_1) (Heatmap med udvikling over residential og commercial)
    - [TITEL Figure 2](#Figur_2) (RESIDENTIAL VS INDUSTRI BARPLOT)
    - [TITEL Figure 3](#Figur_3) (COMMERCIAL UDVIKLING I FORHOLD TIL BOUROUGHS)
    - [TITEL Figure 4](#Figur_4) (Den generelle udvikling af populationen i New York i forhold til Borough)
    - [TITEL Figure 5](#Figur_5) (UDVIKLINGEN I FORHOLD TIL FOREIGN POPULATION BY BOROUGH)
    - [TITEL Figure 6](#Figur_6) (Heatmap over fordelingen af forskellige ethniciteter i NYC)
    - [TITEL Figure 7](#Figur_7) (choropleth over de forskellige etnicitetsfordelinger)
6. [Discussion](#Discussion)
7. [Contribution](#Controbution)
8. [References](#References)

To run the notebook you need to load the packages in the next section

In [ ]:
import geopandas as gpd
import requests 
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams["font.family"] = "Garamond"
import numpy as np
import pandas as pd
import seaborn as sns
import requests
import folium 
from folium import plugins
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as mcolors
import plotly.graph_objs as go
import json
from jinja2 import Template
from folium.map import Layer
from branca.element import Template, MacroElement

# Motivation <a id="Motivation"></a>
- What is your dataset?
- Why did you choose this/these particular dataset(s)?
- What was your goal for the end user's experience?


Let's paint the picture! A boat filled with tired but hopeful families glides across the still morning waters. They have all left their old homes for the promises and opportunities of the new world. In the distance, through the mist, a statue the size of a skyscraper breaks through the morning fog. The mood among the seaworn travellers brightens, realizing that they have finally arrived at their destination. Soon after, a skyline of actual skyscrapers paints the horizon and the bustling sounds of city life fill the air. Sounds familiar?

New York City is undeniably one of - if not *the* most influential and renowned city in western society. In many ways, the city incapsulates the idea The American Dream, with its buzzing streets, opportune business life, and rich cultural scene, spawning sayings like *"if you can make it here, you can make it anywhere"*. Pop cultural references to New York City is omnipresent in western movies, music, and art and each year [millions of turists](https://en.wikipedia.org/wiki/List_of_cities_by_international_visitors) from all over the world flock to the city see all the "familiar" places in real life. *The City that never sleeps*, *the Capital of the World*, *the Big Apple* - New York City has earned itself many nicknames. The City has the [second highest GDP in the world](http://www.citymayors.com/statistics/richest-cities-2020.html), and continues to be [the largest city in the USA for over 200 years](https://www.newyorkfed.org/medialibrary/media/research/epr/05v11n2/0512glae.pdf) with a population of around 8.5 million people today. Professor of economics at Harvard University, Edward L. Glaeser, states that *"While Boston's history is one of ongoing crises and reinvention (Glaeser 2005), New York's is one of almost unbroken triumph"* (Glaeser 2005:1). But what could be the explanation for NYC's urbanizational triumph the last 200 years?

Just as the history of hopeful immigrants arriving to New York City on boats from Europe is a central part of the city's identity, so are it's iconic skyscraper. The physical appearance of the city have come to shape it's economic, social and cultural fabric. Take for instance this very popular picture from 1932 of a band of construction workers taking their lunch break on a beam floating high above the streets of New York. The picture on the right shows the characters from the hugely popular sitcome *Friends*, doing more of less the same thing.

<div style="display: flex; justify-content: center;">
  <img style="width: 29%; height: 30%; margin-right: 1%;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Lunch_atop_a_Skyscraper_-_Charles_Clyde_Ebbets.jpg/1280px-Lunch_atop_a_Skyscraper_-_Charles_Clyde_Ebbets.jpg">
  <img style="width: 40%; height: 10%; margin-left: 1%;" src="https://i.pinimg.com/originals/e5/7a/29/e57a29d29399922bd29bc220ce794a42.jpg">
</div>

The history of how New York City came to be The Big Apple - economicaly, socially, and culturally - thus have to include the history of how the city's developed it's physical form. 


Answering this question is a complex task, but we seek to give the reader a more granular understanding of the urbanisational development of NYC by using visualizations and statistics, and as a result maybe also come a closer to answering the question of NYC’s 200-year triumph. We hope that by presenting these essential parts of NYC history we can show patterns in the overall history of New York thus pushing the boundaries for urban knowledge. We will map two important parts of development in the history of NYC, its commercializational development and its migrational development. We look into the commercialisational development of NYC, as we want to understand better how NYC has developed to become such a rich city with the second highest GDP in the world. We look into migrational development as a lot of the cultural and historical identity of NYC is inadvertently tied to its status as a city build by migrants (https://www.osc.state.ny.us/files/reports/osdc/pdf/report-7-2016.pdf). Thus we hope by visualizing patterns in the urbanisational development of NYC with commercialization and migration as our points of interest, we become able to see the historical effects through time, while understanding contemporary NYC better. 
 In this project we start by BLABLBALBLBAALBA



# Basic Statistics <a id="base_stats"></a>
- Write about your choices in data cleaning and preprocessing
- Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis.

In this project we use multiple datasets as to get the most nuanced and representative data on New York cities development through time with focus on segregation and migration. '

In this section on Basic statistics we have three different headings, each representing af different dimension in which for which we have constructed a data set. All our datasets come from trustworthy different US government organisations and institutions. 

To be able to construct our maps of NYC we have found a geojson file with the mappings for the different boroughs of NYC as to make our visualisations possible. This we preload, before anything else. 

In [ ]:
# NYC Borough GeoJson
new_york_boroughs_map = gpd.read_file("https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/new-york-city-boroughs.geojson")

### **Dataset 1: Land Use in New York <a id="Dataset_of_the_buildings"></a>**
Overvejelser til afsnit: 
    - Skal vi også præsentere GeoJson dataset for NYC hbegrænsningerer, eller skal man måske gøre det endnu mere i starten?  

One of the essential sources of data for this project is the Primary Land Use Tax Lot Output (PLUTO) data from [NYC Planning](https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page) which has data on the use of all of the land plots of NYC. This dataset gives us information on the use of the land plot (*landuse*) and year for when the construction of the buildings on the landplots was completed(*yearbuilt*).  Using this dataset we can map the development of NYC from 1800-2020 start to finish.

The PLUTO dataset also has some limitations. One limitation is that the dataset only have data for buildings and land plots used today. Thus we do not have information on for example buildings which were demolished and rebuild or whether the land was once residential and then changed to commercial. A second limitation is regarding the variable *yearbuilt*, were it is important to be skeptical as the data dictionary states that the buildings construction was not neccesarily done on that specific indicated year, as within the timeframe 1910 and 1985 a majority of the construction years are in years ending in 5 or 0 (Pluto Data Dictionary: 35). A third limitation is that a large amount of buildings which was build between 1800 and the early 1900s are stated to be between 1899 and 1901 (Pluto Data Dictionary: 35). Acknowledging these limitations we still argue that the data represent an opportunity to see New York evolve through time.  

In the following code we load and clean the data so it is ready for analysis.

In [ ]:
# PLUTO Data from https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page
columns_subset = ["borough","cd", "latitude", "longitude", "yearbuilt", 'landuse', "assesstot", "numbldgs",
                  "numfloors", "unitstotal", "bldgarea", "comarea", "resarea", "bbl"]

land_use_dataaset = pd.read_csv("Exam_datasets/pluto_22v3_1.csv")[columns_subset]
land_use_dataaset.shape

Before the data cleaning process the dataset has a total of 858.619 rows with 14 columns. 

##### Cleaning PLUTO data 

In [ ]:
# drop nan for PLUTO
land_use_dataaset = land_use_dataaset.dropna().copy() # drop na 
land_use_dataaset.bbl = land_use_dataaset.bbl.apply(round).astype(str)
land_use_dataaset = land_use_dataaset.query("yearbuilt >= 1600").copy()
land_use_dataaset.yearbuilt = land_use_dataaset.yearbuilt.astype(int)

# Construct bins 
ten_year_bins = [0]+[year for year in range(1899,2029,10)]+[2030]
ten_year_labels = ["Before 1900"] + [str(year)+"s" for year in range(1900,2020,10)] + ["2020s"]

# Into Bins
land_use_dataaset["yearbuilt_intervals"] = pd.cut(land_use_dataaset.yearbuilt, bins = ten_year_bins, labels = ten_year_labels)

# Dicts to convert values  
landuse_key = {1:"One & Two Family Building",
                2:"Multi-Family Walk-Up Buildings",
                3:"Multi-Family Elevator Buildings",
                4:"Mixed Residential & Commercial Buildings",
                5:"Commerical & Office Buildings",
                6:"Industrial & Manufacturing Buildings",
                7:"Transportation & Utility",
                8:"Public Facilities & Institutions",
                9:"Open Space & Outdoor Recreation",
                10: "Parking Facilities",
                11:"Vacant Land"}

borough_key = {"BK":"Brooklyn",
               "QN":"Queens",
               "MN":"Manhattan",
               "BX":"Bronx",
               "SI":"Staten Island"}

# Map dicts
land_use_dataaset.borough = land_use_dataaset.borough.apply(lambda x: borough_key[x])
land_use_dataaset["landuse_label"] =  land_use_dataaset.landuse.apply(lambda x: landuse_key[x])

OBS: Hvornår skal vi fjerne disse? 

In [ ]:
# Removed early observations - not much happening in these years
land_use_dataaset_trimmed = land_use_dataaset.query("yearbuilt >= 1800").copy()

In [ ]:
print(f"We end with a final dataset with a total of {land_use_dataaset.shape[0]} observations and a total of {land_use_dataaset.shape[1]} columns")

After the cleaning and preparation process we have final dataset with a total of XXXXXX plots of land and 15 different columns. This means we have removed approximately 50.000 observations. 

One of the visualization we are going to present using the PLUTO data, is a temporal heatmap of the geographical distribution of construction in New York City including a distinction between residential land use and commercial/non-residential land use. Below, we wrangle the PLUTO data to allow for such analysis.  

In [ ]:
# For mixed landuse, clasify as either Mostly Commerical or Mostly Residential based on square feet 
land_use_dataaset_trimmed["land_use_for_heatmap"] = np.where((land_use_dataaset_trimmed.landuse_label == "Mixed Residential & Commercial Buildings") &\
                                                                 (land_use_dataaset_trimmed.comarea > land_use_dataaset_trimmed.resarea),
                                                                 "Mostly Commerical",
                                                                  land_use_dataaset_trimmed.landuse_label)

land_use_dataaset_trimmed["land_use_for_heatmap"] = land_use_dataaset_trimmed["land_use_for_heatmap"].replace("Mixed Residential & Commercial Buildings", "Mostly Residential")

As mentioned above, the exact year of construction is associated with a certain degree of uncertainty, Thus, we group the construction years into 5 year bins, so e.g. the years 1993-1997 are grouped into 1995. Likewise, the year 1988-1992 are grouped into 1990. 

In [ ]:
# group year to the nearest multple of 5 (rounding down, e.g. 1997->1995)
land_use_dataaset_trimmed['yearbuilt_grouped'] = round(land_use_dataaset_trimmed['yearbuilt'] / 5) * 5

We split PLUTO into a residentail and a commercial/non-residential based on primary land use

In [ ]:
## Subset based on residential and non-residential land-use 
residential_buildings_and_landuse = land_use_dataaset_trimmed[land_use_dataaset_trimmed.land_use_for_heatmap.isin(['One & Two Family Building',
                                                                                                                    "Mostly Residential",
                                                                                                                    'Multi-Family Walk-Up Buildings',
                                                                                                                    'Multi-Family Elevator Buildings',
                                                                                                                    ])].copy()

commercial_buildings_and_landuse = land_use_dataaset_trimmed[land_use_dataaset_trimmed.land_use_for_heatmap.isin(['Mostly Commercial',
                                                                                                                    "Commerical & Office Buildings",
                                                                                                                    'Industrial & Manufacturing Buildings',
                                                                                                                    'Transportation & Utility',
                                                                                                                    "Parking Facilities",
                                                                                                                    "Public Facilities & Institutions"
                                                                                                                    ])].copy()

In [ ]:
print(f"Lots used primarily for commercial/industrial purposes: {commercial_buildings_and_landuse.shape[0]}")
print(f"Lots used primarily for residential purposes: {residential_buildings_and_landuse.shape[0]}")

There are quite a few more residential lots than commercial/industrial ones. Let's look at the relationship when looking at the total number of square feet. 

In [ ]:
print(f"Billion square feet for commercial/industrial use: {commercial_buildings_and_landuse.bldgarea.sum() / 1_000_000_000}")
print(f"Billion square feet for residential use: {residential_buildings_and_landuse.bldgarea.sum() / 1_000_000_000}")

Judging by square feet, we see that the are about 2.5 times more residential space than commercial/industrial. 

Next, we need to divide the data based on the constructions's associated 5-year time bin. 

In [ ]:
# Define data range
data_range = range(min(residential_buildings_and_landuse.yearbuilt.min(), commercial_buildings_and_landuse.yearbuilt.min()),
                   max(residential_buildings_and_landuse.yearbuilt.max(), commercial_buildings_and_landuse.yearbuilt.max())+1, 5)

residential_list_of_lists_with_coordinates = [residential_buildings_and_landuse.query(f"yearbuilt_grouped == {year}")[["latitude", "longitude"]].values.tolist() for year in data_range]
commercial_list_of_lists_with_coordinates = [commercial_buildings_and_landuse.query(f"yearbuilt_grouped == {year}")[["latitude", "longitude"]].values.tolist() for year in data_range]

data_range

The PLUTO data has now been divided based on primary land use and converted into a list of lists with coordinates, where the index of the coordinate-list represents a 5-year bin between 1800 and 2020. We now have all we need for our temporal heatmap.  

## **Dataset 2: Population Development and Foreign Born Population of New York<a id="Dataset_of_the_population"></a>**

Besides our dataset of the buildings of NYC, another essential part of our analysis is the population evolution of NYC. To further examine the development of the migration and population movements of NYC, we have used data from [the official website of NYC](https://www.nyc.gov/). We have used two datasets from this website, [one on the overall population development](https://www.nyc.gov/assets/planning/download/office/planning-level/nyc-population/historical-population/nyc_total_pop_1900-2010.xlsx) and [one on the development of the foreign born population](https://www.nyc.gov/assets/planning/download/office/planning-level/nyc-population/historical-population/nyc_fb_pop_1900-2010.xlsx). Both datasets are rather similar, with the first one containing the overall population numbers for each NYC borough each decade, and the second containing the amount of foreing born population in each NYC borough each decade.

### **Dataset 2.1: Overall Population**

##### Loading and preparing the datasets

In [ ]:
population_by_borough = pd.read_excel("https://www.nyc.gov/assets/planning/download/office/planning-level/nyc-population/historical-population/nyc_total_pop_1900-2010.xlsx", skiprows = 3, index_col = 0)
population_by_borough = population_by_borough.dropna().reset_index(names = ["decade"])

Get population data **from 2020** from US Census Bureau's Decinnal Census: https://www.census.gov/data/developers/data-sets/decennial-census.html 

In [ ]:
# US Census parameters 

API_KEY = "ffe97aa3a40b95750950c76a41624538483d4731"

NY_STATE = ','.join(["36"])

COUNTIES = ','.join(["047", "061", "005", "081", "085"])

COUNTY_TO_BOROUGH = {"081": "Queens",
                     "085": "Staten Island",
                     "047": "Brooklyn",
                     "005": "Bronx",
                     "061": "Manhattan"}

POP_LABEL = {"P1_001N":"population"}

In [ ]:
url = f"https://api.census.gov/data/2020/dec/pl?get=NAME,P1_001N&for=county:{COUNTIES}&in=state:{NY_STATE}&key={API_KEY}"
resp = requests.get(url).json()

NYC_pop_2020 = pd.DataFrame(resp[1:], columns = resp[0])

# Clean data 
NYC_pop_2020["BOROUGH"] = NYC_pop_2020.county.map(COUNTY_TO_BOROUGH)
NYC_pop_2020 = NYC_pop_2020.rename(columns = POP_LABEL)
NYC_pop_2020["population"] = NYC_pop_2020["population"].astype(float) 
NYC_pop_2020 = NYC_pop_2020.drop(["NAME", "state", "county"], axis = 1)
NYC_pop_2020["decade"] = 2020
NYC_pop_2020 = NYC_pop_2020.pivot(index = "decade", values= "population", columns = "BOROUGH")
NYC_pop_2020["New York City"] = NYC_pop_2020.values.sum()
NYC_pop_2020 = NYC_pop_2020.reset_index()

Concatenate 2020 data with 1900-2010 data

In [ ]:
population_by_borough = pd.concat([population_by_borough,NYC_pop_2020]).set_index("decade")

Get population estimates from NYC Planning (Manual extraction from PDF): https://www.nyc.gov/assets/planning/download/pdf/planning-level/nyc-population/projections_report_2010_2040.pdf

Note: the projections were made in 2013. 

In [ ]:
NYC_pop_projection = {"New York City":{2030:8_821_027,
                                       2040:9_025_145},
                      "Bronx":{2030:1_518_998, 
                               2040:1_579_245},
                      "Brooklyn":{2030:2_754_009,
                                  2040:2_840_525},
                      "Manhattan":{2030:1_676_720,
                                   2040:1_691_617},
                      "Queens":{2030:2_373_551,
                                2040:2_412_649},
                      "Staten Island":{2030:497_749,
                                       2040:501_109}}

NYC_pop_projection = pd.DataFrame(NYC_pop_projection)

# Concat 2020 data to make the lines in the plot have the same offset 
NYC_pop_2020 = population_by_borough.loc[2020:]
NYC_pop_projection = pd.concat([NYC_pop_2020, NYC_pop_projection])

### **Dataset 2.2: Foreign Born Population of New York**  

In [ ]:
foreign_born_pop = pd.read_excel("https://www.nyc.gov/assets/planning/download/office/planning-level/nyc-population/historical-population/nyc_fb_pop_1900-2010.xlsx", skiprows = 3, index_col = 0)
foreign_born_pop = foreign_born_pop.dropna().reset_index(names = ["decade"])
foreign_born_pop.decade = foreign_born_pop.decade.apply(lambda x: str(x).replace("*","")).astype(int)
foreign_born_pop = foreign_born_pop.set_index("decade")

Calculate the share of foreign born population within each borough since 1900

In [ ]:
share_of_foreign_born_by_decade = foreign_born_pop / population_by_borough.loc[1900:2010] * 100
share_of_foreign_born_by_decade = share_of_foreign_born_by_decade.drop("New York City", axis =1 )

### **Thought on Dataset 2.1 and 2.2**

Because of the simplicity of both datasets their is not much cleaning or preparation to take care of since both datasets consists of 12 rows and 7 columns. The construction of the datasets seems to have been a rather complicated endeavour with combinations of data form multiple sources, and it would probably be possible to write a full data science project regarding the composition of the datasets which presents exact population numbers for each decade over a 110 year period. We decide to trust the NYC government and the data legibility, but if one would like to scrutinise this decision it would be possible to start going through the sources used for each dataset her: [documentation for population development](https://www.nyc.gov/assets/planning/download/pdf/planning-level/nyc-population/historical-population/nyc_total_pop_1900-2010.pdf) and [documentation for foreign born population development](https://www.nyc.gov/assets/planning/download/pdf/planning-level/nyc-population/historical-population/nyc_fb_pop_1900-2010.pdf)

### **Dataset 3: The Ethnic Population in New York City Neighboorhoods**

Our third and final dataset we use to map the ethnicity throughout contemporary NYC. This data we got using [the United States Census Bureaus API](#https://www.census.gov/data/developers/data-sets.html), this API is rather large with numerous dataset on all of The United States of America. We use a dataset gathered as part of the [American Community Survey (ACS)](#https://www.census.gov/programs-surveys/acs), they gather data on the communities of USA to help officials. ACS have four surveys dataset: [1-year estimates, 1-year supplemental estimates, 3-year estimates and 5-year estimates](#https://www.census.gov/programs-surveys/acs/guidance/estimates.html). For this project we use the [5-year estimates](#https://www.census.gov/data/developers/data-sets/acs-5year.html) because it gives the most reliable and granular data. The 5-year estimates dataset is a very big survey for all of USA, while at the same time being granular to block group level, which we specifically use to investigate the contemporary neighbourhood compositions of NYC. We are interested in the *race* variables of the survey on a neighbourhood level, but one should also be a bit cautious. The *race* variable is selfreport and self-identification, so people themselves can choose the group which they identify with, and they can choose multiple ethnicity groups (https://www.census.gov/topics/population/race.html). Thus we for example found the *race* variable to give more than 100% of the population in each neighbourhood. As we only want an estimate of the ethnicity spread of the neighborhoods of NYC we argue the data still probably represent the overall ethnic tendencies of the neighborhoods. 

For using the API it is necessary to get an Key, and also to navigate the different code. As we are only interested in NY state and in that state we are only interested NYC we had to both specify states and counties. In the following code we call the API and clean the data.

### **Dataset 3.1: Data on Ethnicity**

In [ ]:
COLUMN_LABELS = {"DP05_0001E": "total_pop",
                 "DP05_0003PE": "pct_female",
                 "DP05_0002PE": "pct_male",
                 "DP03_0002PE": "pct_employed_over_16",
                 "DP03_0005PE": "pct_unemployed_over_16",
                 "DP03_0052PE": "pct_below_10000_income", # der er mange inkomstgrupperinger - Overvej om de skal bruges 
                 "DP03_0062E": "median_household_income",
                 "DP03_0063E" : "also_median_household_income?",
                 "DP05_0037PE": "pct_white_one_race",
                 "DP05_0038PE": "pct_black_one_race",
                 "DP05_0044PE": "pct_asian_one_race",
                 "DP05_0071PE": "pct_hispanic_or_latino_any",
                 "DP05_0086E": "total_housing_units"}

COUNTY_TO_BOROUGH = {"081": "Queens",
                     "085": "Staten Island",
                     "047": "Brooklyn",
                     "005": "Bronx",
                     "061": "Manhattan"}


SEX_VAR = ["DP05_0003PE","DP05_0002PE"]
POP_VAR = ["DP05_0001E"]
LABOUR_VAR = ["DP03_0002PE", "DP03_0005PE"]
INCOME_VAR = ["DP03_0052PE", "DP03_0062E","DP03_0063E"]
RACE_VAR = ["DP05_0037PE", "DP05_0038PE", "DP05_0044PE", "DP05_0071PE"]
HOUSING_VAR = ["DP05_0086E"]

QUERY = ",".join(SEX_VAR+POP_VAR+LABOUR_VAR+INCOME_VAR+RACE_VAR+HOUSING_VAR)

In [ ]:
url = f"https://api.census.gov/data/2021/acs/acs5/profile?get=NAME,{QUERY}&for=tract:*&in=county:{COUNTIES}&in=state:{NY_STATE}&key={API_KEY}"
resp = requests.get(url).json()
us_census_df = pd.DataFrame(resp[1:], columns = resp[0])

# Clean response 
us_census_df = us_census_df.rename(columns = COLUMN_LABELS)
us_census_df["BOROUGH"] = us_census_df.county.map(COUNTY_TO_BOROUGH)

us_census_df = us_census_df.query("tract!='990100'") # Remove "errorneus" tracts
us_census_df[list(COLUMN_LABELS.values())] = us_census_df[list(COLUMN_LABELS.values())].astype(float) # Convert to float
us_census_df = us_census_df.replace(-666666666, np.nan) # Convert nan values 

### I areas where noone lives (based on the "total population") replace Nan-value in the other columns with 0. 
for col in list(COLUMN_LABELS.values()):
    us_census_df[col] = np.where((us_census_df["total_pop"] == 0 & pd.isna(us_census_df[col])),
                                 0,
                                 us_census_df[col])

### **Dataset 3.2: Geodata on Census Tract Level**

Load GeoJson with Census Tract geometry

In [ ]:
census_tracts_geo = gpd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Census_Tracts_for_2020_US_Census/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")

Align with the dataset from the US Census Bereau

In [ ]:
## Match US Census data tract IDs to the census tract geo data IDs
US_CENSUS_KEY = census_tracts_geo[["BoroName", "BoroCode"]].drop_duplicates().set_index("BoroName").to_dict()["BoroCode"]
us_census_df["borough_id"] = us_census_df.BOROUGH.map(US_CENSUS_KEY).astype("str")
us_census_df["tract_id"] = us_census_df["borough_id"] + us_census_df["tract"]

## Subset columns and rename 
census_tracts_geo = census_tracts_geo[["NTAName", "CDTANAME", "BoroCT2020", "geometry"]].rename(columns={"BoroCT2020":"tract_id"}).copy()

Merge geometry on Census data

In [ ]:
census_tract_data = pd.merge(census_tracts_geo, us_census_df, on = "tract_id", how = "outer", indicator = True)

Sanity Check on merge

In [ ]:
census_tract_data._merge.value_counts() # All tracts with demographic data are in the merge 

Examine the tract that could not be merged

In [ ]:
census_tract_data.query("_merge == 'left_only'")

`Hoffman & Swinburne Islands` are two small and uninhabited island off the coast of Staten Island. Thus, filtering these out will not affect the remaining demographic analysis.  

Keep only observations in both dataset 

In [ ]:
census_tract_data = census_tract_data.query("_merge == 'both'")

KONKLUSION, MEN ER LIDT I TVIVL OM HVORVIDT DET HER ER DATASÆTTET

# **Data Analysis <a id="Data_Analysis"></a>**
- Describe your data analysis and explain what you've learned about the dataset.
- If relevant, talk about your machine-learning.

# **Genre <a id="Genre"></a>**
- Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?
- Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?

When constructing a narrative it is important to be clear and prepare ones "data story" (Segel & Heer 2010: 1). One way to do this, is to decide on on the visual narrative and the narrative structure. 

Overall our website and data story structure is rather linear, with most in common with the magazine visualization genre described by Segel and Heer (Segel & Heer 2010:7). We take starting point in our wordplay "from seed to apple", this we do by starting with a broader look at the historical data and 'coming' of NYC's development. Thus we use timebars which go back to 1900 to examine and visualise the historical development of NYC. Timebars are practical to give a framing for ones visual narrative. 

As we also are concerned with keeping the reader active and engaged, we incorporate interactive plots in which the reader can explore and engage within the frames of our data story and magazine style, thus if the reader has an interest in one certain year it is be possible to stop the visualisation and look closer. This magasine style genre will have a narrative structure which are linear as to follow our overall idear with mapping New York from seed to apple.

# **Visualizations<a id="Visualizations"></a>**

In this section, we plot and explain the visualization we have chosen to include in our data story. For each visualization, we start by presenting the code needed for the given vizualization, which naturally concludes with plotting the visualization. Then, we present our thought behind our choice of visualization and its contribution to the overall narrative. 

We have an overall of X visualizations in our project. 

### **Visualization 1: The Temporal and Geographical Development of New York City's Residential and Commercial Body<a id="Figur_1"></a>**

First, we define a function that creates a colormap the is compatible with Plotly's `HeatMapWithTime`, so we can distinguish between residential and commercial/industrial construction.

In [ ]:
def color_gradient_heatmap(hexcode, n_gradients):
    ''' Create colormap from 'White' to the specified hexcode in with n_gradients ''' 
    _cmap = mcolors.LinearSegmentedColormap.from_list(
    "Custom",["#FFFFFF", hexcode], N=n_gradients
    )

    gradient_dict = {i/n_gradients:mcolors.rgb2hex(_cmap(i)) for i in range(n_gradients+1)}

    return gradient_dict

As the `HeatMapWithTime` does not have an inbuilt method for distinguishing between two cateogries/groups of datapoints (residential vs. non-residential) we have to tweek the module a bit. Below, we define the class `HeatMapWithTimeAdditional` which allows us to add an extra layer in the `HeatMapWithTime`, where both the main and additional layer share the same control bar / slider. Thanks to @Conengmo for providing a solution to this, in the following github-issue: https://github.com/python-visualization/folium/issues/1062

In [ ]:
class HeatMapWithTimeAdditional(Layer):
    _template = Template("""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = new TDHeatmap({{ this.data }},
                {heatmapOptions: {
                    radius: {{this.radius}},
                    minOpacity: {{this.min_opacity}},
                    maxOpacity: {{this.max_opacity}},
                    scaleRadius: {{this.scale_radius}},
                    useLocalExtrema: {{this.use_local_extrema}},
                    defaultWeight: 1,
                    {% if this.gradient %}gradient: {{ this.gradient }}{% endif %}
                }
            }).addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)

    def __init__(self, data, name=None, radius=15,
                 min_opacity=0, max_opacity=0.6,
                 scale_radius=False, gradient=None, use_local_extrema=False,
                 overlay=True, control=True, show=True):
        super(HeatMapWithTimeAdditional, self).__init__(
            name=name, overlay=overlay, control=control, show=show
        )
        self._name = 'HeatMap'
        self.data = data

        # Heatmap settings.
        self.radius = radius
        self.min_opacity = min_opacity
        self.max_opacity = max_opacity
        self.scale_radius = 'true' if scale_radius else 'false'
        self.use_local_extrema = 'true' if use_local_extrema else 'false'
        self.gradient = gradient

Aside from the additional layer, we also need to do some HTML/CSS tweaking to add a legend to the plot. Thanks to Tile Mile Documentation here: https://tilemill-project.github.io/tilemill/docs/guides/advanced-legends/

In [ ]:
legend_template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; left: 12px; top: 460px;'>
     
<div class='legend-title'>Primary Lot Use</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FF2B2B;opacity:0.7;height:15px;width:15px;border-radius:50%';display:block;float:left;align-items:center;></span>Residential Construction</li>
    <li><span style='background:#2BB2FF;opacity:0.7;height:15px;width:15px;border-radius:50%';display:block;float:left;align-items:center;></span>Commercial Construction</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

Finally, we are mostly setup. Below, we plot the geograprical and temporal development of New York City's residential and commercial/industrial body. The red markers represent residential construction while the blue markers are commercial/industrial construction. We have also outlined the 5 boroughs of New York City: Manhattan, Brooklyn, Bronx, Queens, and Staten Island. 

**Please Note**: Per default, the plot is paused and the `layers` that shows residential and commercial construction are both turned off. Turn the layers on through the layer-control in the top right corner and press play to initiate the animation. The division into layer allows the reader to explore the  various aspects of the plot, if interested (ja, nu snakker jeg bare... men vi skal være eksplicit omkring at de skal vælge de der `layers` til). 

In [ ]:
## Params 
radius = 5
opacity = 0.8
blur = 0.8
show = False
overlay = True


# Create NYC map 
NY_coord = [40.690610, -73.935242]
NY_map = folium.Map(NY_coord,
                    width=1000, 
                    height=600,
                    tiles =None,
                    zoom_start = 10,
                    min_zoom = 10,
                    max_lat = 41,
                    min_lat = 40.3,
                    max_lon = -73.3,
                    min_lon = -74.5,
                    max_bounds = True)

# Name base layer
folium.TileLayer('cartodbpositron', name='NYC Buildings').add_to(NY_map)

# Add legend from template
macro = MacroElement()
macro._template = Template(legend_template)
NY_map.get_root().add_child(macro)

# # Cmap for commercial buildings 
commercial_gradient = color_gradient_heatmap("#2BB2FF", 100)
residential_gradient = color_gradient_heatmap("#FF2B2B", 100)

## Residential Heatmap
plugins.HeatMapWithTime(residential_list_of_lists_with_coordinates,
                        index = [f"Year: {year}" for year in data_range],
                        auto_play = False,
                        max_opacity=opacity,
                        radius = radius,
                        blur = blur,
                        overlay = overlay,
                        show = show,
                        name = "Residential Buildings",
                        gradient = residential_gradient,
                        ).add_to(NY_map)

## Add Commercial Heatmap 
HeatMapWithTimeAdditional(commercial_list_of_lists_with_coordinates,
                            radius = radius,
                            max_opacity=opacity,
                            overlay = overlay,
                            show = show, 
                            name = "Commerical Buildings",
                            gradient = commercial_gradient).add_to(NY_map)



# # add neighborhoods
folium.GeoJson(new_york_boroughs_map[["geometry", "name"]],
                name = "Bourough",
                tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['Bourough:']),
                style_function =  lambda x: {"fillColor":"#00BCFF" if x["properties"]["name"]=="Staten Island" else \
                                                            "#FFAA00" if x["properties"]["name"]=="Queens" else \
                                                            "#FF0000" if x["properties"]["name"]=="Brooklyn" else \
                                                            "#FFFFFF" if x["properties"]["name"]=="Manhattan" else \
                                                            "#B4FF00" if x["properties"]["name"]=="Bronx" else "",
                                                            "color":"black",
                                                            "weight":1}).add_to(NY_map)

folium.LayerControl().add_to(NY_map)

# Show plot 
#NY_map

In [ ]:
# map_html = NY_map._repr_html_()

# #  Creating an HTML file
# Func = open("Plots/construction_heatmap.html","w")
   
# # Adding input data to the HTML file
# Func.write(map_html)
              
# # Saving the data into the HTML file
# Func.close()

### **Thoughts on Visualization 1**
The plot is meant to give a historical introduction of New York City while also giving the reader a geographical understanding of New York City and its five boroughs. 


The heatmap quickly becomes quite cluttered and at times, one can hardly spot anything of the underlying map simply due to the excessive degree of construction. There is a quit interesting analytical point to this. The plot also makes it clear how New York City's journey from seed to apple started on Manhatten, slowly growing into Brooklyn, rapidly spreading throughout the 1920 and 1930, ultimatly taking up all available space in the 5 borough, resulting in some of the urbanizational problems that most major cities know of today, with social segregation, sky-rocketing housing prices and upwards being the only feasible direction for further construction. 


As previously mentioned, the uncertainty associated with the dataset becomes apparent with the "boom" in construction starting in 1900 - either because of faulty records prior to this point, or because uncertainty regarding build year was handled by recording the construction year as some-round-number e.g. 1900. 

Talk about encoding! Not the best here to get a quantitative and comperative impression of the data. However, for the geographical distribution and for overall introductory purposes, we find that the heatmap to be well suited, and eye-catching - Drawing the reader in. 


Again, **remember turn the layers on** through the layer-control in the top right corner and press play to initiate the animation.

### **TITEL Figure 2 () (RESIDENTIAL VS INDUSTRI BARPLOT)<a id="Figur_2"></a>**

In [ ]:
fig, ax  = plt.subplots(figsize = (10,6))

residential_squarefeet_by_decade = land_use_dataaset_trimmed.groupby("yearbuilt_intervals").resarea.sum().sort_index()
commercial_squarefeet_by_decade = land_use_dataaset_trimmed.groupby("yearbuilt_intervals").comarea.sum().sort_index()

residential_squarefeet_by_decade.plot.bar(ax=ax, color = "#E9655C",  alpha = 0.8, width = 0.8)
commercial_squarefeet_by_decade.plot.bar(ax=ax, bottom = residential_squarefeet_by_decade, alpha = 0.8, color = "skyblue", width = 0.8)

ax.yaxis.get_offset_text().set_visible(False) # Removes the scientific notation
ax.set_title("Squarefeet by Decade", size = 22)
ax.set_ylabel("Square Feet (in hundred millions)", size = 20)
ax.set_xlabel("")
ax.legend(["Residential", "Commercial"], fontsize = "x-large")
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

## Thoughts on Figure 2

- The Booming '20 
- Small recession in the 1930 - maybe the aftermath of the Wall Street Crash in 1929, giving the decade a rough start. 
- 1940 are characterized by WW2. 
- 1960 are also known as the "happy sixties".
- Energy crisis in 1970.

The relation between residential and commerical construction looks quite stable, somewhere between 1/2 and 1/6 across the years. 
-   **MERE HEROM**

The physical development of NYC is clearly affect by the world around it. Energy crises, wars, and economic recession are all manifested in the plot above. 


With the geographical understanding of the positioning of the five boroughs, let's take a deeper dive into the construction of New York City from a residential vs. industrial perspective. 

### **TITEL FIGURE 3 (COMMERCIAL UDVIKLING I FORHOLD TIL BOUROUGHS)**

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(20, 6), sharey=True)
fig.suptitle('Square Feet by Decade and Borough (in hundred millions)', fontsize=22, y=1.05)

categories = land_use_dataaset_trimmed['borough'].unique()

for i, category in enumerate(categories):
    ax = axs[i]
 
    residential_squarefeet_by_decade = land_use_dataaset_trimmed[land_use_dataaset_trimmed['borough'] == category].groupby("yearbuilt_intervals").resarea.sum().sort_index()
    commercial_squarefeet_by_decade = land_use_dataaset_trimmed[land_use_dataaset_trimmed['borough'] == category].groupby("yearbuilt_intervals").comarea.sum().sort_index()

    ax.bar(residential_squarefeet_by_decade.index, residential_squarefeet_by_decade.values, color="#E9655C", alpha=0.8, width=0.8)
    ax.bar(commercial_squarefeet_by_decade.index, commercial_squarefeet_by_decade.values, bottom=residential_squarefeet_by_decade.values, color="skyblue", alpha=0.8, width=0.8)
    ax.yaxis.get_offset_text().set_visible(False) # Removes the scientific notation

    ax.set_title(category, size=18)
    ax.set_xlabel("")
    ax.set_xticks(np.arange(0, len(residential_squarefeet_by_decade.index), 1))
    ax.set_xticklabels(residential_squarefeet_by_decade.index, rotation=90, fontsize=12)

    if i == 0:
        ax.set_ylabel("Square Feet", size=20)
        ax.legend(["Residential", "Commercial"], fontsize="x-large")
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()


## Thoughts on Figure 3

- Staten Island:
    - The construction of bridges allowed for a new wave of commuters. 
    - Mostely Residential
    - relatively later than the other areas
- Bronx: 
    - Mostly Residential 

- Brooklyn:
    - Commerical booom in the 20' and 30' 
    - Part of the early settlement
- Manhattan:
    - Relatively high degree of commerical construction. Commerical boom in the 1920' 
    - Part of the early settlement
- Queens:
    - construction remained relatively high and stable throughout the 1920-1960, i.e. during war time. 

# TEST: FLOORS 

In [ ]:
floor_bins = [-np.inf, 1, 10, np.inf]
floor_labels = ["Single Plan", "2-10 floors", "+10 floors"]
land_use_dataaset_trimmed["numfloors_cat"] = pd.cut(land_use_dataaset_trimmed.numfloors, bins = floor_bins, labels = floor_labels, right = True)

In [ ]:
fig, ax = plt.subplots(5,1, figsize = (10,15))
floor_plot = land_use_dataaset_trimmed[["yearbuilt_intervals","borough","numfloors_cat"]].value_counts().rename("count").reset_index()

for idx, borough in enumerate(floor_plot['borough'].unique()):
    borough_data = floor_plot.query(f"borough == '{borough}'").copy()
    sns.barplot(borough_data, x="yearbuilt_intervals", y = "count", hue = "numfloors_cat", errorbar = None, ax = ax[idx])
    ax[idx].set_title(borough)
    
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(20, 6), sharey=True)
fig.suptitle('Square Feet by Decade and Borough (in hundred millions)', fontsize=22, y=1.05)

categories = land_use_dataaset_trimmed['borough'].unique()

for i, category in enumerate(categories):
    ax = axs[i]
 
    residential_squarefeet_by_decade = land_use_dataaset_trimmed[land_use_dataaset_trimmed['borough'] == category].groupby("yearbuilt_intervals").resarea.sum().sort_index()
    commercial_squarefeet_by_decade = land_use_dataaset_trimmed[land_use_dataaset_trimmed['borough'] == category].groupby("yearbuilt_intervals").comarea.sum().sort_index()

    ax.bar(residential_squarefeet_by_decade.index, residential_squarefeet_by_decade.values, color="#E9655C", alpha=0.8, width=0.8)
    ax.bar(commercial_squarefeet_by_decade.index, commercial_squarefeet_by_decade.values, bottom=residential_squarefeet_by_decade.values, color="skyblue", alpha=0.8, width=0.8)
    ax.yaxis.get_offset_text().set_visible(False) # Removes the scientific notation

    ax.set_title(category, size=18)
    ax.set_xlabel("")
    ax.set_xticks(np.arange(0, len(residential_squarefeet_by_decade.index), 1))
    ax.set_xticklabels(residential_squarefeet_by_decade.index, rotation=90, fontsize=12)

    if i == 0:
        ax.set_ylabel("Square Feet", size=20)
        ax.legend(["Residential", "Commercial"], fontsize="x-large")
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

### **TITEL Figure 4 (Den generelle udvikling af populationen i New York i forhold til Borough)<a id="Figur_4"></a>**

In [ ]:
# https://www.nyc.gov/assets/planning/download/pdf/planning-level/nyc-population/projections_report_2010_2040.pdf (Found from this report from NYC)
NYC_pop_projection = {"New York City":{2030:8_821_027,
                                       2040:9_025_145},
                      "Bronx":{2030:1_518_998, 
                               2040:1_579_245},
                      "Brooklyn":{2030:2_754_009,
                                  2040:2_840_525},
                      "Manhattan":{2030:1_676_720,
                                   2040:1_691_617},
                      "Queens":{2030:2_373_551,
                                2040:2_412_649},
                      "Staten Island":{2030:497_749,
                                       2040:501_109}}

NYC_pop_projection = pd.DataFrame(NYC_pop_projection)

# Concat 2020 data to make the lines in the plot have the same offset 
NYC_pop_2020 = population_by_borough.loc[2020:]
NYC_pop_projection = pd.concat([NYC_pop_2020, NYC_pop_projection])

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))

colors = ["#000000", "#2BAAFF", "#FFAA00", "#FF0000", "#A15C03", "#18B406"]

population_by_borough.plot(color= colors, ax=ax, legend = False, alpha = 0.7)
NYC_pop_projection.plot(color = colors, style="--", legend = False, ax=ax, alpha = 0.7)

ax.set_title("Population by Borough", size = 20)
ax.set_ylabel("Population (in millions)", size = 18)
ax.set_xlabel("Decade", size = 15)
plt.xticks(list(population_by_borough.index) + [2030, 2040], fontsize=15)
plt.yticks(fontsize=15)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.axvline(2020, linestyle = "--", linewidth = 1, c="gray", alpha = 0.5)

# Change y-ticks
ax.set_yticklabels([int(tick/1000000) for tick in ax.get_yticks()])

# Annotate
ax.text(2040, NYC_pop_projection.loc[2040,'New York City'], 'New York\n     City', size=12, color="#000000")
ax.text(2041, NYC_pop_projection.loc[2040,"Bronx"]-150_000, 'Bronx', size=12, color="#2BAAFF")
ax.text(2041, NYC_pop_projection.loc[2040,"Brooklyn"], 'Brooklyn', size=12, color="#FFAA00")
ax.text(2041, NYC_pop_projection.loc[2040,"Manhattan"]+150_000, 'Manhattan', size=12, color="#FF0000")
ax.text(2041, NYC_pop_projection.loc[2040,"Queens"], 'Queens', size=12, color="#A15C03")
ax.text(2041, NYC_pop_projection.loc[2040,"Staten Island"]-200_000, 'Staten\nIsland', size=12, color="#18B406")
# Text and Arrow for population projection 
ax.text(2023, 5_000_000, 'Population\nprojection', size=12, color='gray')
ax.annotate("", xy=(2038, 4_700_000), xytext=(2022.5, 4_700_000),
            arrowprops=dict(arrowstyle="->", color ="gray", alpha=0.7))

plt.savefig("Plots/population_by_borough.png", dpi = 300, bbox_inches = "tight")

plt.tight_layout()
plt.show()

### **Thoughts on figure 4**

Aside from a decrease in the population during 1970-2000, New York City has had a constantly growing number of inhabitants with the most rapid increase in the early 2000th century. Based on the population projection, this trend seem to continue going forward to 2040.

Manhattan:
-  Peak population in 1910s.
Brooklyn:
-   Peaked around 1950, but almost at the same level again in 2020. 

Queens:
- Constantly increasing, overtaking Bronx between 1940 and 1950, and Manhattan during the 1950's. 

Staten Island:
- Slowly increasing, though not at all near the other boroughs. 


WHY DECREASING DURING 1970-1990?

### **TITEL Figure 5 (UDVIKLINGEN I FORHOLD TIL FOREIGN POPULATION BY BOROUGH)<a id="Figur_5"></a>**

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

fig.suptitle("Foreign Born Population by Borough from 1900-2010", size=20)
## Foreign born population
colors = ["#000000", "#2BAAFF", "#FFAA00", "#FF0000", "#A15C03", "#18B406"]
foreign_born_pop.plot(color=colors, ax=axs[0], legend=False, alpha=0.7)
axs[0].set_ylabel("Foreign Born Population (in millions)", size=18)
axs[0].set_xticks(share_of_foreign_born_by_decade.index)
axs[0].set_xlabel("", size=15)
axs[0].yaxis.get_offset_text().set_visible(False) # Removes the scientific notation
#axs[0].set_yticklabels(['%.1f' % float(tick/1000000)+ ' mil.' for tick in axs[0].get_yticks()])
axs[0].set_xticklabels("")
axs[0].tick_params(axis='both', which='major', labelsize=15)
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

# Annotate
axs[0].text(2011, foreign_born_pop.loc[2010,'New York City'], 'New York\n     City', size=12, color=colors[0])
axs[0].text(2011, foreign_born_pop.loc[2010,"Bronx"]+50_000, 'Bronx', size=12, color=colors[1])
axs[0].text(2011, foreign_born_pop.loc[2010,"Brooklyn"]-30_000, 'Brooklyn', size=12, color=colors[2])
axs[0].text(2011, foreign_born_pop.loc[2010,"Manhattan"]-50_000, 'Manhattan', size=12, color=colors[3])
axs[0].text(2011, foreign_born_pop.loc[2010,"Queens"], 'Queens', size=12, color=colors[4])
axs[0].text(2011, foreign_born_pop.loc[2010,"Staten Island"]-50_000, 'Staten\nIsland', size=12, color=colors[5])

## Share of foreignborn in population
colors = ["#2BAAFF", "#FFAA00", "#FF0000", "#A15C03", "#18B406"]
share_of_foreign_born_by_decade.plot(color=colors, ax=axs[1], legend=False, alpha=0.7)
axs[1].set_title("", size=20)
axs[1].set_ylabel("Share of Population", size=18)
axs[1].set_xlabel("Decade", size=15)
axs[1].set_xticks(share_of_foreign_born_by_decade.index)
axs[1].tick_params(axis='both', which='major', labelsize=15)
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

# Annotate
axs[1].text(2011, share_of_foreign_born_by_decade.loc[2010,"Bronx"], 'Bronx', size=12, color=colors[1-1])
axs[1].text(2011, share_of_foreign_born_by_decade.loc[2010,"Brooklyn"], 'Brooklyn', size=12, color=colors[2-1])
axs[1].text(2011, share_of_foreign_born_by_decade.loc[2010,"Manhattan"], 'Manhattan', size=12, color=colors[3-1])
axs[1].text(2011, share_of_foreign_born_by_decade.loc[2010,"Queens"], 'Queens', size=12, color=colors[4-1])
axs[1].text(2011, share_of_foreign_born_by_decade.loc[2010,"Staten Island"], 'Staten\nIsland', size=12, color=colors[5-1])

plt.savefig("Plots/foreign_And_share_population_by_borough.png", dpi = 300, bbox_inches = "tight")
plt.tight_layout()
plt.show()


## **Thoughts on Figure 6**

On the website: Callback to the familiar picture of immigrant ariving to New York and talk about identity. 


Increasing -1930, then decreasing all the way up into 1970 (maybe incentivised by the Immigration Act thing, @Asger). 

Manhattan:
- Had the clearly highest share of foreign born inhabitaten up to and including 1910, but the share dropped rapidly from 1910-1970.
- The borough with the least steep increase in its share if foreign borns from 1970 up until 2010. All others have increase quite a bit. 
- Only borough to have decline in its share of foreign borns between decades since 1970. 

Queens:
- Queens have continously increased its amount of foreign born inhabitants. Still decreased in it's share between 1900-1950, indicated that others have moved to queens as well. 
- Booming since 1950s




### **TITEL Figure 6 (Heatmap over fordelingen af forskellige ethniciteter i NYC) <a id="Figur_6"></a>**

### **Explanation for choosen visualisations**

In [ ]:
fig, ax = plt.subplots(2,2, figsize = (10,10))

# Create a choropleth map
census_tract_data.plot(column='pct_white_one_race', cmap='YlOrBr', edgecolor = "grey", linewidth=0.0, ax = ax[0][0])
new_york_boroughs_map.plot(facecolor = "none", edgecolor = "black", linewidth = 0.1, ax = ax[0][0])
ax[0][0].set_axis_off()

# Add a colorbar
cbar = ax[0][0].get_figure().colorbar(ax[0][0].collections[0], shrink = 0.5, location='bottom', pad = 0)
cbar.set_label('Pct. White Residents', labelpad=0)

census_tract_data.plot(column='pct_black_one_race', cmap='YlOrBr', ax = ax[0][1])
new_york_boroughs_map.plot(facecolor = "none", edgecolor = "black", linewidth = 0.1, ax = ax[0][1])
ax[0][1].set_axis_off()
# Add a colorbar
cbar = ax[0][1].get_figure().colorbar(ax[0][1].collections[0], shrink = 0.5, location='bottom', pad = 0)
cbar.set_label('Pct. Black Residents', labelpad = 0)

census_tract_data.plot(column='pct_asian_one_race', cmap='YlOrBr', ax = ax[1][0])
new_york_boroughs_map.plot(facecolor = "none", edgecolor = "black", linewidth = 0.1, ax = ax[1][0])
ax[1][0].set_axis_off()
# Add a colorbar
cbar = ax[1][0].get_figure().colorbar(ax[1][0].collections[0], shrink = 0.5, location='bottom', pad = 0)
cbar.set_label('Pct. Asian Residents', labelpad = 0)

census_tract_data.plot(column='pct_hispanic_or_latino_any', cmap='YlOrBr', ax = ax[1][1])
new_york_boroughs_map.plot(facecolor = "none", edgecolor = "black", linewidth = 0.1, ax = ax[1][1])
ax[1][1].set_axis_off()
# Add a colorbar
cbar = ax[1][1].get_figure().colorbar(ax[1][1].collections[0], shrink = 0.5, location='bottom', pad = 0)
cbar.set_label('Pct. Hispanic/Latino Residents', labelpad = 0)

# Save 
plt.savefig("Plots/race_dist_by_tract.png", dpi = 300, bbox_inches = "tight")

# Show the map
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

# Create a choropleth map
census_tract_data.plot(column='median_household_income', cmap='YlOrBr', edgecolor = "grey", linewidth=0.0, ax = ax)
new_york_boroughs_map.plot(facecolor = "none", edgecolor = "black", linewidth = 0.1, ax = ax)
ax.set_axis_off()

### **Thoughts on Figure 6**

Clear clustering. 

Staten Island and Manhattan are remarkably white boroughs - excluding China Town in Manhattan. 

### **TITEL Figure 7 (Choropleth over fordelingen af forskellige ethniciteter i NYC) <a id="Figur_7"></a>**

In [ ]:
census_tract_data["hovertext"] = "<br>Neighborhood: " + census_tract_data.NTAName +\
                                 "<br>Borough: " + census_tract_data.BOROUGH +\
                                 "<br>Total population: " + census_tract_data.total_pop.apply(round).astype(str)

In [ ]:
def cmap_with_alpha(alpha=0.7, n_colors=100, plt_cmap='YlOrBr'):
    cmap_rgba_vals = plt.get_cmap(plt_cmap, n_colors) # get rgba values from the plt_cmap
    cmap_rgb_vals = [tuple(map(lambda x: x*255, cmap_rgba_vals(i)[:3])) for i in range(n_colors)][5:] # convert to rgb + slice to exclude white as lower color
    GEOMAP_CMAP = [f"rgba{rgb+tuple([alpha])}" for rgb in cmap_rgb_vals] # set alpha and convert to plotly-compatible format
    return GEOMAP_CMAP

In [ ]:
label_dict = {'pct_white_one_race':"Pct. White Residents",
              'pct_black_one_race':"Pct. Black Residents",
              'pct_asian_one_race':"Pct. Asian Residents",
              'pct_hispanic_or_latino_any':"Pct. Hispanic & Latino Residents"}

# we need to add this to select which trace 
# is going to be visible
visible = np.array(list(label_dict.keys()))

# define traces and buttons at once
traces = []
buttons = []
for col, label in label_dict.items():

    traces.append(go.Choroplethmapbox(geojson=json.loads(census_tract_data.to_json()),
                                    z = census_tract_data[col],
                                    locations = census_tract_data.index,
                                    colorscale=cmap_with_alpha(),
                                    #colorbar_title = label,
                                    colorbar=dict(
                                        title=label,
                                        titleside='top',
                                        len = 0.8,
                                        orientation = "h",
                                        y = -0.17,
                                        tickfont = dict(size=15) # set the position of the colorbar title
                                    ),
                                    marker_line_width=0.1,
                                    visible= True if col==list(label_dict.keys())[0] else False,
                                    text = census_tract_data["hovertext"],
                                    hovertemplate= f"{label}: " + "%{z}%"+
                                                    "%{text}<extra></extra>"
                                    ))

    buttons.append(dict(label=label,
                        method="update",
                        args=[{"visible":list(visible==col)},
                              {"title":""}]))

updatemenus = [{"active":0,
                "buttons":buttons,
                "direction":'down',
                "showactive":True,
                "x":0.02,
                "y":0.98,
                "xanchor":"left",
                "yanchor":"top"}]


# Show figure
fig = go.Figure(data=traces,
                layout=dict(updatemenus=updatemenus))

fig.layout['title']['text'] = ""
                  


fig.update_layout(mapbox_style="carto-positron",
                    height = 600,
                    width = 1000,
                    autosize=True,
                    paper_bgcolor= "#FFF6E9",
                    margin={"r":0,"t":0,"l":0,"b":0},
                    mapbox=dict(center={"lat": 40.690610, "lon": -73.935242},zoom=9),
                    font_family = "Garamond",
                    font_size = 18
                    )

fig.show()

### **Thoughts on Figure 7**

Nogle konkrete eksempler. zoom ind. 

# **Discussion<a id="Discussion"></a>**
- What went well?,
- What is still missing? What could be improved?, Why?

- HER KUNNE MAN EVENTUELT SKRIVE OM LIMITATIONS VED PLUTO? 
- MAN KUNNE OGSÅ FÅR INFRASTRUKTUR MED 
- Eventuelt overveje noget prediction på indvandring? 
- Further Studie: A closer look at critical infrastructure like the construction of Brooklyn Bridge, or the presence of New York Habour for New York's economic development. 
- Housing Prices. 

# **Contributions<a id="Contributions"></a>**
- You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That's what you should explain).
- It is not OK simply to write "All group members contributed equally".

# **References <a id="References"></a>**
- Make sure that you use references when they're needed and follow academic standards.
- Glaeser 2005: https://www.newyorkfed.org/medialibrary/media/research/epr/05v11n2/0512glae.pdf

Pluto data dictionary: https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page

# **Datasets**

- Building footprints, [City of New York](https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh) (SKAL FJERNES LIGE NU!)
- PLUTO data, [NYC Planning](https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page)
- [The overall population development of NYC's boroughs](https://www.nyc.gov/assets/planning/download/office/planning-level/nyc-population/historical-population/nyc_total_pop_1900-2010.xlsx) 
- [The overall development of the foreign born population of NYC's boroughs](https://www.nyc.gov/assets/planning/download/office/planning-level/nyc-population/historical-population/nyc_fb_pop_1900-2010.xlsx)

# GAMMELT

Our dataset of the buildings and the related plot of land is essential for our initial analysis and illustration of New York Cities evolution throughout time. This dataset is a combination of two datasets: One dataset from [City of New York](https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh) on the building footprints of New York, and another dataset from [NYC Planning](https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page) on the land plots of NYC. In the following code we load, clean, and merge the two datasets. 

In [ ]:
# ## Loading the two datas

# # Building footprints from https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh
# # Documentation https://github.com/CityOfNewYork/nyc-geo-metadata/blob/master/Metadata/Metadata_BuildingFootprints.md
# building_footprints = gpd.read_file("Exam_datasets/Building_Footprints.geojson") 

# # NYC Borough GeoJson
# new_york_boroughs_map = gpd.read_file("https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/new-york-city-boroughs.geojson")

# # PLUTO Data from https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page
# columns_subset = ["borough","cd", "latitude", "longitude", 'landuse', "assesstot", "numbldgs",
#                   "numfloors", "unitstotal", "bldgarea", "comarea", "resarea", "bbl"]
# land_use_dataaset = pd.read_csv("Exam_datasets/pluto_22v3_1.csv")[columns_subset]

In [ ]:
# buildings_and_landuse = pd.merge(building_footprints, land_use_dataaset, on = "bbl", how = "outer", indicator = True)

In [ ]:
# print("Total amount of unique buildings in the dataset: {}".format(len(buildings_and_landuse)))
# print("Total amount of unique land plots: {}".format(buildings_and_landuse["bbl"].nunique()))
# print("The dataset has {} columns".format(len(buildings_and_landuse.columns)))

We combine the two dataset based on BBL column which stand for "borough, block and lot", merging on this variabe makes it possible to combine our data on buildings with our data on land lots.Before cleaning the dataset consists of 31 columns with a total of 10.898.686 buildings and a total 817.842 unique land plots. Furthermore we get info on how the buildings land plots is used, so we get a distinction between whether a building is build on commercial or residential land. 

This is not the final data, as we first do a sanity check of the merge and check how many BBL values from each of the two datasets does not exist in the other. 

##### Sanity check on merge between Buildings and land plots

In [ ]:
# # We construct two explorative datasets to explore how many buildings which only exist in one of the two datasets
# only_in_building_footprints = buildings_and_landuse.query("_merge == 'left_only'")
# only_in_pluto = buildings_and_landuse.query("_merge == 'right_only'")

# print(f"Buildings not found in Building Footprint: {only_in_pluto.shape[0]}")
# print(f"Buildings not found in PLUTO: {only_in_building_footprints.shape[0]}")

With a dataset consisting of a total of 10.898.686 observations, some missing values seems to be unavoidable. Looking into the amount of missing values we have a total of 20.443 missing values which correspond to 0.2 percent of the total dataset. We think this is rather low, but even a low amount of missing values can be a problem if biased towards a certain group or category, therefore we will look into the distribution of the missing values.

In [ ]:
# # Create a figure with two subplots
# fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4))

# # Plot the first bar plot onto the first subplot
# only_in_pluto.borough.value_counts().plot.bar(title="Distribution of missing buildings from the Building Footprints dataset", ax=ax1)

# # Plot the second bar plot onto the second subplot
# buildings_and_landuse.borough.value_counts().plot.bar(title="Distribution of buildings in the main dataset", ax=ax2)

# # Set a common y-axis label for both subplots
# #fig.text(0.04, 0.5, va='center', rotation='vertical')

# # Display the plot
# plt.tight_layout()
# plt.show()

The Building Footprints data has a total of 6.033 missing values. We look into into the distribution of these missing values among the five boroughs of NYC, and conclude that the overall distribution of missing values look rather evenly spread. It is apparent that Queens and Brooklyn is the boroughs with the highest amount of missing values, but this shows an unbias distribution of missing values as these two also are the boroughs with the overall highest amount of buildings.   

In [ ]:
# # Create a figure with two subplots
# fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 4))

# # Plot the first bar plot onto the first subplot
# only_in_building_footprints.cnstrct_yr_intervals.value_counts().sort_index().plot.bar(title = "Distribution of missing buildings not found in PLUTO", ax=ax1)

# # Plot the second bar plot onto the second subplot
# buildings_and_landuse.cnstrct_yr_intervals.value_counts().sort_index().plot.bar(title = "Distribution of buildings in the main dataset", ax=ax2)

# # Set a common y-axis label for both subplots
# #fig.text(0.04, 0.5, va='center', rotation='vertical')

# # Display the plot
# plt.tight_layout()
# plt.show()

In [ ]:
# buildings_and_landuse = buildings_and_landuse.query("_merge == 'both'").copy()
# buildings_and_landuse.cnstrct_yr = buildings_and_landuse.cnstrct_yr.astype("int64")
# # Dimensions of dataset 
# buildings_and_landuse.shape

This has been a presentation of one of the three essential datasets used in this project, the buildings and land use dataset. We have explained the overall dataset, and argue that the amount of missing data when combining the two datasets is miniscule and unbiased. After cleaning and merging the overall dataset we have a datasaet containing 31 columns with 1.069.453 rows of unique buildings and their related landplot.  

In [ ]:
# import matplotlib.gridspec as gridspec

# # Create a 2x1 grid for the top plot and the bottom row of subplots
# fig = plt.figure(figsize=(20, 12))
# gs = gridspec.GridSpec(nrows=2, ncols=1, height_ratios=[1, 1])

# # Create the top plot
# ax1 = fig.add_subplot(gs[0])
# residential_squarefeet_by_decade = land_use_dataaset_trimmed.groupby("yearbuilt_intervals").resarea.sum().sort_index()
# commercial_squarefeet_by_decade = land_use_dataaset_trimmed.groupby("yearbuilt_intervals").comarea.sum().sort_index()
# residential_squarefeet_by_decade.plot.bar(ax=ax1, color="#E9655C", alpha=0.8, width=0.8)
# commercial_squarefeet_by_decade.plot.bar(ax=ax1, bottom=residential_squarefeet_by_decade, alpha=0.8, color="skyblue", width=0.8)
# ax1.set_title("Squarefeet by Decade in New York ", size=22)
# ax1.set_ylabel("Square Feet (in hundred millions)", size=20)
# ax1.set_xlabel("")
# ax1.yaxis.get_offset_text().set_visible(False) # Removes the scientific notation
# ax1.legend(["Residential", "Commercial"], fontsize="x-large")
# ax1.tick_params(axis='y', labelsize=15) # Only change ytick font size
# ax1.spines['top'].set_visible(False)
# ax1.spines['right'].set_visible(False)

# # Create a 1x5 grid for the subplots
# gs_subplots = gridspec.GridSpecFromSubplotSpec(nrows=1, ncols=5, subplot_spec=gs[1], wspace=0.1)

# categories = land_use_dataaset_trimmed['borough'].unique()

# # Find the maximum value of residential and commercial square feet across all boroughs
# max_residential = land_use_dataaset_trimmed.groupby("yearbuilt_intervals").resarea.sum().max()
# max_commercial = land_use_dataaset_trimmed.groupby("yearbuilt_intervals").comarea.sum().max()
# y_max = max(max_residential, max_commercial)

# for i, category in enumerate(categories):
#     ax = fig.add_subplot(gs_subplots[i], sharey=ax1)

#     residential_squarefeet_by_decade = land_use_dataaset_trimmed[land_use_dataaset_trimmed['borough'] == category].groupby("yearbuilt_intervals").resarea.sum().sort_index()
#     commercial_squarefeet_by_decade = land_use_dataaset_trimmed[land_use_dataaset_trimmed['borough'] == category].groupby("yearbuilt_intervals").comarea.sum().sort_index()

#     ax.bar(residential_squarefeet_by_decade.index, residential_squarefeet_by_decade.values, color="#E9655C", alpha=0.8, width=0.8)
#     ax.bar(commercial_squarefeet_by_decade.index, commercial_squarefeet_by_decade.values, bottom=residential_squarefeet_by_decade.values, color="skyblue", alpha=0.8, width=0.8)
#     ax.yaxis.get_offset_text().set_visible(False) # Removes the scientific notation

#     ax.set_title(category, size=18)
#     ax.set_xlabel("")
#     ax.set_xticks(np.arange(0, len(residential_squarefeet_by_decade.index), 1))
#     ax.set_xticklabels(residential_squarefeet_by_decade.index, rotation=90, fontsize=12)

#     if i == 0:
#         ax.set_ylabel("Square Feet (in hundred millions)", size=20)
# #        ax.legend(["Residential", "Commercial"], fontsize="x-large") # Add legend 
    
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)

# plt.tight_layout()
# plt.show()
